# Tensor decomposition median Week

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import tensorly as tl
import time
import copy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colrs

import pickle as pkl

from tqdm import tqdm
import torch
import gc

In [2]:
tl.set_backend('pytorch')

os.environ['CUDA_DEVICE_ORDER']= 'PCI_BUS_ID' # see issue #152
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
USER = os.getlogin()
WORKING_DIR = f'/home/{USER}/data/Land_use'
DATA_DIR = f'{WORKING_DIR}/data'
IMAGE_DIR = f'{WORKING_DIR}/images'
CITY_NAME = 'Paris'

In [4]:
rank = [5, 5, 5]

if len(sys.argv) > 4:
    if sys.argv[1] == '-my_params':
    
        rank = [int(sys.argv[2]), int(sys.argv[3]), int(sys.argv[4])]

In [2]:
np.random.seed(42)
rank_values = []
for rank_space in range(3, 13):
    for rank_app in range(3, 13):
        for rank_time in range(3, 13):

            rank_values.append((rank_space, rank_app, rank_time))

#selected_ranks = np.random.choice(len(rank_values), 2000, replace=False)

# for ranks in selected_ranks:
#     rank_space, rank_app, rank_time = rank_values[ranks]
#     line = f'python /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.py -my_params {rank_space} {rank_app} {rank_time} >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log'
#     print(line)

In [3]:
# select_ranks = rank_values[5430:]

for ranks in rank_values:
    rank_space, rank_app, rank_time = ranks
    line = f'python /home/jupyter-landy/data/Land_use/4.0-save_time_factors_against_rank.py -my_params {rank_space} {rank_app} {rank_time} >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log'
    # line = f'python /home/orlando_martinez/data/Land_use/2.1-Tucker_decomposition_explore_ranks_gpu_2.py -my_params {rank_space} {rank_app} {rank_time} >> /home/orlando_martinez/data/Land_use/2.1-Tucker_decomposition_explore_ranks_gpu_2.log'
    print(line)

python /home/jupyter-landy/data/Land_use/4.0-save_time_factors_against_rank.py -my_params 3 3 3 >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log
python /home/jupyter-landy/data/Land_use/4.0-save_time_factors_against_rank.py -my_params 3 3 4 >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log
python /home/jupyter-landy/data/Land_use/4.0-save_time_factors_against_rank.py -my_params 3 3 5 >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log
python /home/jupyter-landy/data/Land_use/4.0-save_time_factors_against_rank.py -my_params 3 3 6 >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log
python /home/jupyter-landy/data/Land_use/4.0-save_time_factors_against_rank.py -my_params 3 3 7 >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log
python /home/jupyter-landy/data/Land_use/4.0-save_time_factors_against_rank.py -my_params 3 3 8 >> /home/jupyter-landy/d

In [ ]:
# smaller_ranks = []
# higher_ranks = []
# for ranks in rank_values[667:]:
#     rank_space, rank_app, rank_time = rank_values[ranks]
#     if rank_space + rank_app + rank_time < 30:
#         smaller_ranks.append((rank_space, rank_app, rank_time))
#     else:
#         higher_ranks.append((rank_space, rank_app, rank_time))

# # for ranks in smaller_ranks:
# #     rank_space, rank_app, rank_time = ranks
# #     line = f'python /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.py -my_params {rank_space} {rank_app} {rank_time} >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log'
# #     print(line)

# for ranks in higher_ranks[700:]:
#     rank_space, rank_app, rank_time = ranks
#     line = f'python /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.py -my_params {rank_space} {rank_app} {rank_time} >> /home/jupyter-landy/data/Land_use/2.1-Tucker_decomposition_explore_ranks.log'
#     print(line)

## Load Data Tensor

In [ ]:
fd = open(f'{DATA_DIR}/tensor.pkl', 'rb')
numpy_tensor = pkl.load(fd)
numpy_tensor = numpy_tensor[0]
fd.close()

In [ ]:
# smaller_tensor = numpy_tensor[230:230+5000, :, :]

## Decomposition

In [ ]:
start_time = time.time()

tensorly_tensor = tl.tensor(numpy_tensor, device='cuda', dtype=torch.float32)
core, factors = tl.decomposition.tucker(tensorly_tensor, rank=rank)
end_time = time.time()
delta_time = (end_time - start_time)

tensorly_tensor_approx = tl.tucker_to_tensor((core, factors))
mse_tensor = tl.metrics.regression.MSE(tensorly_tensor, tensorly_tensor_approx)
rmse_tensor = tl.metrics.regression.RMSE(tensorly_tensor, tensorly_tensor_approx)
# congruence, _ = tl.metrics.congruence_coefficient(tensorly_tensor, tensorly_tensor_approx)

numpy_tensor_approx = tensorly_tensor_approx.cpu().numpy()
l2_norm_error = np.linalg.norm(numpy_tensor - numpy_tensor_approx)

print(f'{rank[0]}\t{rank[1]}\t{rank[2]}\t{delta_time:.2f}\t{mse_tensor:.4f}\t{rmse_tensor:.4f}\t{l2_norm_error:.4f}')